## Calculate Human-only Results (CBCC)

In [1]:
import sys, os
import pandas as pd
import polars as pl

In [2]:
sys.path.append(f"../")

In [3]:
from agg_methods_cbcc import get_aggregation_methods_CBCC

In [4]:
from io_utils import get_accuracy, get_recall, load_dataset_profile, load_gt, load_human_responses

In [6]:
def get_save_file_path(dataset_profile, r):
    dataset_name = dataset_profile["dataset_name"]
    n_classes = dataset_profile["n_classes"]

    file_name = f"../results_human_cbcc/{dataset_name}_{r}.csv"
    if os.path.exists(file_name):
        assert False, "File already exists"
    with open(file_name, "w") as f:
        f.write("dataset,scenario,method,ai_acc,r,num_ai,iter,convergence,accuracy,recall,uc_p,")
        for i in range(n_classes):
            f.write(f"uc_pih_{i},")
        for i in range(n_classes-1):
            f.write(f"uc_pia_{i},")
        f.write(f"uc_pia_{i+1}")
        f.write("\n")
    return file_name

In [7]:
DATSETS = ["dog", "face", "tiny", "adult"]

In [8]:
SEEDS = [12347, 77777, 2984, 298, 1102644]

In [ ]:
for dataset in DATSETS:
    print(f"Dataset: {dataset}")
    dataset_profile = load_dataset_profile(dataset)
    gt, biased_tasks = load_gt(dataset)

    methods = get_aggregation_methods_CBCC(dataset_profile["labels"])
    for r in dataset_profile["r_range"]:
        file_name = get_save_file_path(dataset_profile, r)
        human_responses = load_human_responses(dataset, r)
        ai_responses = human_responses.iloc[0:0].copy()
        print(f"File will be saved to: {file_name}")
        for iter in range(5):
            for method in methods:
                print(f"Method: {method.name}")
                ret = method.fit_predict(human_responses, ai_responses, seed=SEEDS[iter])
                acc = get_accuracy(gt, ret)
                biased_acc = get_recall(gt, ret, biased_tasks)
                print(f"Accuracy: {acc}, Recall: {biased_acc}")
                uc_text = method.get_uc_text(dataset_profile["n_classes"])
                with open(file_name, "a") as f:
                    f.write(f"{dataset},*,{method.name},*,{r},0,{iter},-1,{acc},{biased_acc},{uc_text}\n")